## Imports

In [ ]:
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import cv2
import os
import sys

## MSE Function

In [ ]:
def calc_mse(im1, im2):
    err = np.sum((im1.astype("float") - im2.astype("float")) ** 2)
    err /= float(im1.shape[0] * im1.shape[1])
    
    return err

## Settings

In [ ]:
im1_dir = '/home/drevital/obstacles_classification_datasets/rgb_6_H/eval/obstacle'
im1_name = '169_1908__enhanced__blur_1.jpg'
search_dir = '/home/drevital/obstacles_classification_datasets/rgb_6_H/train/no_obstacle'
out_dir = '/home/drevital/obstacles_classification_datasets/rgb_6_H/sims'
im_height = 200
im_width = 400
min_thresh = 0.00
max_thresh = 1.00
min_mse_thresh = 0.10
max_mse_thresh = 0.20
max_sim = -1
most_sim = None
most_sim_mse = 0

## Read Image

In [ ]:
im1_rgb = cv2.imread(os.path.join(im1_dir, im1_name))
im1 = cv2.cvtColor(im1_rgb, cv2.COLOR_BGR2RGB) 

## Prepare Image for Compaison

In [ ]:
im1 = cv2.resize(im1,
                 dsize=(im_width, im_height),
                 interpolation=cv2.INTER_LINEAR)
im1 = np.array(im1, dtype='float32')
w = im1.shape[1]
im1 = im1[:, w//2:]
im1 /= 255.0

## Search for Similarities

In [ ]:
sims = []
sim_names = []
sim_scores = []
fnames = os.listdir(search_dir)
most_sim_name = None

for fname in tqdm(fnames):
    im2_rgb = cv2.imread(os.path.join(search_dir, fname))
    im2 = cv2.cvtColor(im2_rgb, cv2.COLOR_BGR2RGB) 
    im2 = cv2.resize(im2,
                     dsize=(im_width, im_height),
                     interpolation=cv2.INTER_LINEAR)
    im2 = np.array(im2, dtype='float32')
    w = im2.shape[1]
    im2 = im2[:, w//2:]
    im2 /= 255.0
    similarity = ssim(im1, im2, multichannel=True)
    mse = calc_mse(im1, im2)
    if max_thresh >= similarity >= min_thresh and min_mse_thresh <= mse <= max_mse_thresh:
        sim_names.append(fname)
        sims.append(im2)
        sim_scores.append(similarity)
        if similarity > max_sim:
            max_sim = similarity
            most_sim_name = fname
            most_sim = im2
            most_sim_mse = mse

In [ ]:
len(sim_names), len(sims), len(sim_scores)

## Write similar images to output directory

In [ ]:
for i, sim in enumerate(sims):    
    im = cv2.cvtColor(sim, cv2.COLOR_RGB2BGR) 
    im *= 255.0
    prefix = '_'.join(sim_names[i].split('_')[:2])
    imname = f'{prefix}_{sim_scores[i]:.3f}.jpg'
    cv2.imwrite(os.path.join(out_dir, imname), im)

In [ ]:
max_sim, most_sim_mse, most_sim_name

## Display Compared Image

In [ ]:
plt.imshow(im1)
plt.title('Image 1')
plt.show() 